In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel
from langchain_core.tracers.context import collect_runs
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

## Chaining invocations

In [5]:
prompt = PromptTemplate(
    template="Tell me a joke about {topic}"
)

In [6]:
parser = StrOutputParser()

In [7]:
parser.invoke(
    llm.invoke(
        prompt.invoke(
            {"topic": "Python"}
        )
    )
)

'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!'

## Runnables

Runnables can be 
- executed
    - invoke(), 
    - batch() 
    - and stream()
- inspected,
- and composed

In [8]:
runnables = [prompt, llm, parser]

**Execute methods**

In [9]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINVOKE: {repr(runnable.invoke)}")
    print(f"\tBATCH: {repr(runnable.batch)}")
    print(f"\tSTREAM: {repr(runnable.stream)}\n")

PromptTemplate
	INVOKE: <bound method BasePromptTemplate.invoke of PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')>
	BATCH: <bound method Runnable.batch of PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')>
	STREAM: <bound method Runnable.stream of PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')>

ChatOpenAI
	INVOKE: <bound method BaseChatModel.invoke of ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<open

**Inspect**

In [10]:
for runnable in runnables:
    print(f"{repr(runnable).split('(')[0]}")
    print(f"\tINPUT: {repr(runnable.get_input_schema())}")
    print(f"\tOUTPUT: {repr(runnable.get_output_schema())}")
    print(f"\tCONFIG: {repr(runnable.config_schema())}\n")

PromptTemplate
	INPUT: <class 'langchain_core.utils.pydantic.PromptInput'>
	OUTPUT: <class 'langchain_core.prompts.prompt.PromptTemplateOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.PromptTemplateConfig'>

ChatOpenAI
	INPUT: <class 'langchain_openai.chat_models.base.ChatOpenAIInput'>
	OUTPUT: <class 'langchain_openai.chat_models.base.ChatOpenAIOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.ChatOpenAIConfig'>

StrOutputParser
	INPUT: <class 'langchain_core.output_parsers.string.StrOutputParserInput'>
	OUTPUT: <class 'langchain_core.output_parsers.string.StrOutputParserOutput'>
	CONFIG: <class 'langchain_core.utils.pydantic.StrOutputParserConfig'>



In [15]:
runnables[1].config_schema().model_json_schema()

{'properties': {}, 'title': 'ChatOpenAIConfig', 'type': 'object'}

**Config**

In [16]:
with collect_runs() as run_collection:
    result = llm.invoke(
        "Hello", 
        config={
            'run_name': 'demo_run', 
            'tags': ['demo', 'lcel'], 
            'metadata': {'lesson': 2}
        }
    )

In [17]:
run_collection.traced_runs

[RunTree(id=8760039a-5207-43e4-901b-eee81a77bd47, name='demo_run', run_type='llm', dotted_order='20251129T181404878182Z8760039a-5207-43e4-901b-eee81a77bd47')]

In [18]:
run_collection.traced_runs[0].dict()

{'id': UUID('8760039a-5207-43e4-901b-eee81a77bd47'),
 'name': 'demo_run',
 'start_time': datetime.datetime(2025, 11, 29, 18, 14, 4, 878182, tzinfo=datetime.timezone.utc),
 'run_type': 'llm',
 'end_time': datetime.datetime(2025, 11, 29, 18, 14, 6, 734983, tzinfo=datetime.timezone.utc),
 'extra': {'invocation_params': {'model': 'gpt-4o-mini',
   'model_name': 'gpt-4o-mini',
   'stream': False,
   'temperature': 0.0,
   '_type': 'openai-chat',
   'stop': None},
  'options': {'stop': None},
  'batch_size': 1,
  'metadata': {'lesson': 2,
   'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': 0.0}},
 'error': None,
 'serialized': {'lc': 1,
  'type': 'constructor',
  'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
  'kwargs': {'model_name': 'gpt-4o-mini',
   'temperature': 0.0,
   'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']},
   'stream_usage': True},
  'name': 'ChatOpenAI'},
 'events': [{'name': '

**Compose Runnables**

In [19]:
chain = RunnableSequence(prompt, llm, parser)

In [20]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [21]:
chain.invoke({"topic": "Python"})

'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [22]:
for chunk in chain.stream({"topic": "Python"}):
    print(chunk, end="", flush=True)

Why do Python programmers prefer dark mode?

Because light attracts bugs!

In [23]:
chain.batch([
    {"topic": "Python"},
    {"topic": "Data"},
    {"topic": "Machine Learning"},
])

['Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs!',
 'Why did the data break up with the database?\n\nBecause it found too many "issues" in their relationship!',
 'Why did the neural network break up with the decision tree?\n\nBecause it found someone with more layers!']

In [26]:
import grandalf

In [32]:
chain.get_graph().print_ascii()

ImportError: Install grandalf to draw graphs: `pip install grandalf`.

**Turn any function into a runnable**

In [33]:
def double(x:int)->int:
    return 2*x

In [34]:
runnable = RunnableLambda(double)
runnable.invoke(2)

4

**Parallel Runnables**

In [35]:
parallel_chain = RunnableParallel(
    double=RunnableLambda(lambda x: x * 2),
    triple=RunnableLambda(lambda x: x * 3),
)

In [36]:
parallel_chain.invoke(3)

{'double': 6, 'triple': 9}

In [37]:
parallel_chain.get_graph().print_ascii()

ImportError: Install grandalf to draw graphs: `pip install grandalf`.

## LCEL

In [38]:
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')

In [39]:
llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000018695F66270>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000018695F66CF0>, root_client=<openai.OpenAI object at 0x0000018695F64050>, root_async_client=<openai.AsyncOpenAI object at 0x0000018695F66A50>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [40]:
parser

StrOutputParser()

In [41]:
chain = RunnableSequence(prompt, llm, parser)
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000018695F66270>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000018695F66CF0>, root_client=<openai.OpenAI object at 0x0000018695F64050>, root_async_client=<openai.AsyncOpenAI object at 0x0000018695F66A50>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputP

In [42]:
prompt | llm | parser

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}')
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000018695F66270>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000018695F66CF0>, root_client=<openai.OpenAI object at 0x0000018695F64050>, root_async_client=<openai.AsyncOpenAI object at 0x0000018695F66A50>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputP

In [43]:
chain = prompt | llm | parser

In [44]:
chain.invoke(
    {"topic": "computer"}
)

'Why did the computer go to therapy?\n\nBecause it had too many bytes from its past!'